In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

emo_df=pd.read_csv('data/text.csv')
emo_df

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...,...
416804,416804,i feel like telling these horny devils to find...,2
416805,416805,i began to realize that when i was feeling agi...,3
416806,416806,i feel very curious be why previous early dawn...,5
416807,416807,i feel that becuase of the tyranical nature of...,3


In [ ]:
emo_df.drop(columns=['Unnamed: 0'],inplace=True)
# 0: sadness 1:joy 2:love 3:anger 4:fear 5:surprise

In [8]:
from sklearn.model_selection import train_test_split
train_set,val_set=train_test_split(emo_df,test_size=0.2,random_state=42)
train_set.shape,val_set.shape

((333447, 2), (83362, 2))

In [9]:
from tensorflow.keras.layers import TextVectorization,Embedding,Dense,Input

In [10]:
train_inputs=train_set['text'].to_numpy()
val_inputs=val_set['text'].to_numpy()
train_target=train_set['label'].to_numpy()
val_target=val_set['label'].to_numpy()

In [14]:
text_vectorizer=TextVectorization(max_tokens=None,output_mode='int',split='whitespace',ngrams=None,output_sequence_length=None,
                                  standardize='lower_and_strip_punctuation')

In [20]:
sum([len(i.split()) for i in train_inputs])/len(train_inputs)
max_vocab=10000
max_length=20

In [21]:
text_vectorizer=TextVectorization(max_tokens=max_vocab,output_sequence_length=max_length)

In [22]:
text_vectorizer.adapt(train_inputs)

In [23]:
embed=Embedding(input_dim=max_vocab,output_dim=128)

In [24]:
import mlflow
import mlflow.keras

In [31]:
def model_eval(model):
    train_results=model.evaluate(train_inputs,tf.one_hot(train_target,depth=6))
    val_results=model.evaluate(val_inputs,tf.one_hot(val_target,depth=6))
    print(f'Train_Metrics: {train_results}')
    print(f'Test_Metrics: {val_results}')
    train_acc=train_results[1]
    val_acc=val_results[1]
    train_prec=train_results[2]
    val_prec=val_results[2]
    train_rec=train_results[3]
    val_rec=val_results[3]
    mlflow.log_metric('Train_Accuracy',train_acc)
    mlflow.log_metric('Test_Accuracy',val_acc)
    mlflow.log_metric('Train_Precision',train_prec)
    mlflow.log_metric('Test_Precision',val_prec)
    mlflow.log_metric('Train_Recall',train_rec)
    mlflow.log_metric('Test_Recall',val_rec)

In [26]:
from tensorflow.keras.layers import Dense,GRU,LSTM,Bidirectional,Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [27]:
mlflow.set_experiment('Emotion_Detection')

2025/10/22 02:57:08 INFO mlflow.tracking.fluent: Experiment with name 'Emotion_Detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/KM/Coding%21%21%21/vs%20code/Scikit%20Learn/Emotions%20Sentiment%20Analysis/mlruns/546541585463291312', creation_time=1761082028907, experiment_id='546541585463291312', last_update_time=1761082028907, lifecycle_stage='active', name='Emotion_Detection', tags={}>

In [32]:

with mlflow.start_run(run_name='GRU'):
    inputs=Input(shape=(1,),dtype=tf.string)
    x=text_vectorizer(inputs)
    x=embed(x)
    x=GRU(32,activation='tanh',return_sequences=True)(x)
    x=GRU(32,activation='tanh')(x)
    outputs=Dense(6,activation='softmax')(x)
    model_1=Model(inputs,outputs)
    model_1.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy','precision','recall'])
    model_1.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

    mlflow.log_params({'epochs':5,'loss':'CategoricalCrossentropy','optimizer':'Adam','learning_rate':'default',
                       'model_type':'GRU','activation':'tanh','metrics':'Accuracy,Precison,Recall','layers':2,'units':'48,48'})
    model_eval(model_1)

Epoch 1/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 88s 8ms/step - accuracy: 0.8813 - loss: 0.2613 - precision: 0.9248 - recall: 0.8503 - val_accuracy: 0.8671 - val_loss: 0.3047 - val_precision: 0.9063 - val_recall: 0.8479
Epoch 2/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 85s 8ms/step - accuracy: 0.9007 - loss: 0.2022 - precision: 0.9259 - recall: 0.8819 - val_accuracy: 0.8649 - val_loss: 0.3123 - val_precision: 0.9019 - val_recall: 0.8467
Epoch 3/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 90s 9ms/step - accuracy: 0.9076 - loss: 0.1854 - precision: 0.9277 - recall: 0.8923 - val_accuracy: 0.8617 - val_loss: 0.3367 - val_precision: 0.8939 - val_recall: 0.8473
Epoch 4/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 107s 10ms/step - accuracy: 0.9131 - loss: 0.1711 - precision: 0.9296 - recall: 0.9008 - val_accuracy: 0.8621 - val_loss: 0.3734 - val_precision: 0.8860 - val_recall: 0.8512
Epoch 5/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 107s 10ms/step - accuracy: 0.9178 - loss: 0.1589 - precision: 0.9309 - recall: 0.9076 - val_accurac

In [33]:
with mlflow.start_run(run_name='LSTM'):
    inputs=Input(shape=(1,),dtype=tf.string)
    x=text_vectorizer(inputs)
    x=embed(x)
    x=LSTM(48,activation='tanh',return_sequences=True)(x)
    x=LSTM(48,activation='tanh')(x)
    outputs=Dense(6,activation='softmax')(x)
    model_2=Model(inputs,outputs)
    model_2.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy','precision','recall'])
    model_2.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

    mlflow.log_params({'epochs':5,'loss':'CategoricalCrossentropy','optimizer':'Adam','learning_rate':'default',
                       'model_type':'LSTM','activation':'tanh','metrics':'Accuracy,Precison,Recall','layers':2,'units':'48,48'})
    model_eval(model_2)

Epoch 1/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 95s 9ms/step - accuracy: 0.8844 - loss: 0.2526 - precision: 0.9241 - recall: 0.8546 - val_accuracy: 0.8661 - val_loss: 0.3170 - val_precision: 0.8999 - val_recall: 0.8506
Epoch 2/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 91s 9ms/step - accuracy: 0.9065 - loss: 0.1907 - precision: 0.9271 - recall: 0.8901 - val_accuracy: 0.8666 - val_loss: 0.3326 - val_precision: 0.8902 - val_recall: 0.8546
Epoch 3/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 94s 9ms/step - accuracy: 0.9136 - loss: 0.1703 - precision: 0.9294 - recall: 0.9014 - val_accuracy: 0.8638 - val_loss: 0.3503 - val_precision: 0.8885 - val_recall: 0.8524
Epoch 4/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 94s 9ms/step - accuracy: 0.9186 - loss: 0.1554 - precision: 0.9313 - recall: 0.9093 - val_accuracy: 0.8605 - val_loss: 0.3815 - val_precision: 0.8832 - val_recall: 0.8508
Epoch 5/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 91s 9ms/step - accuracy: 0.9239 - loss: 0.1431 - precision: 0.9337 - recall: 0.9162 - val_accuracy: 0

In [34]:
with mlflow.start_run(run_name='Bi_GRU'):
    inputs=Input(shape=(1,),dtype=tf.string)
    x=text_vectorizer(inputs)
    x=embed(x)
    x=Bidirectional(GRU(32,activation='tanh',return_sequences=True))(x)
    x=Bidirectional(GRU(32,activation='tanh'))(x)
    outputs=Dense(6,activation='softmax')(x)
    model_3=Model(inputs,outputs)
    model_3.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy','precision','recall'])
    model_3.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

    mlflow.log_params({'epochs':5,'loss':'CategoricalCrossentropy','optimizer':'Adam','learning_rate':'default',
                       'model_type':'Bi-GRU','activation':'tanh','metrics':'Accuracy,Precison,Recall','layers':2,'units':'32,32'})
    model_eval(model_3)

Epoch 1/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 108s 10ms/step - accuracy: 0.8955 - loss: 0.2249 - precision: 0.9249 - recall: 0.8730 - val_accuracy: 0.8651 - val_loss: 0.3551 - val_precision: 0.8884 - val_recall: 0.8539
Epoch 2/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 114s 11ms/step - accuracy: 0.9127 - loss: 0.1726 - precision: 0.9284 - recall: 0.9008 - val_accuracy: 0.8584 - val_loss: 0.3716 - val_precision: 0.8831 - val_recall: 0.8471
Epoch 3/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 102s 10ms/step - accuracy: 0.9194 - loss: 0.1564 - precision: 0.9312 - recall: 0.9099 - val_accuracy: 0.8614 - val_loss: 0.3971 - val_precision: 0.8795 - val_recall: 0.8527
Epoch 4/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 102s 10ms/step - accuracy: 0.9239 - loss: 0.1441 - precision: 0.9335 - recall: 0.9164 - val_accuracy: 0.8595 - val_loss: 0.4547 - val_precision: 0.8748 - val_recall: 0.8536
Epoch 5/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 121s 12ms/step - accuracy: 0.9270 - loss: 0.1345 - precision: 0.9345 - recall: 0.9211 - val_a

In [35]:
with mlflow.start_run(run_name='Bi_LSTM'):
    inputs=Input(shape=(1,),dtype=tf.string)
    x=text_vectorizer(inputs)
    x=embed(x)
    x=Bidirectional(LSTM(32,activation='tanh',return_sequences=True))(x)
    x=Bidirectional(LSTM(32,activation='tanh'))(x)
    outputs=Dense(6,activation='softmax')(x)
    model_4=Model(inputs,outputs)
    model_4.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy','precision','recall'])
    model_4.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

    mlflow.log_params({'epochs':5,'loss':'CategoricalCrossentropy','optimizer':'Adam','learning_rate':'default',
                       'model_type':'Bi-LSTM','activation':'tanh','metrics':'Accuracy,Precison,Recall','layers':2,'units':'32,32'})
    model_eval(model_4)

Epoch 1/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 100s 9ms/step - accuracy: 0.8912 - loss: 0.2351 - precision: 0.9242 - recall: 0.8656 - val_accuracy: 0.8637 - val_loss: 0.3475 - val_precision: 0.8894 - val_recall: 0.8518
Epoch 2/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 95s 9ms/step - accuracy: 0.9122 - loss: 0.1732 - precision: 0.9288 - recall: 0.8995 - val_accuracy: 0.8609 - val_loss: 0.3645 - val_precision: 0.8824 - val_recall: 0.8504
Epoch 3/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 96s 9ms/step - accuracy: 0.9203 - loss: 0.1533 - precision: 0.9321 - recall: 0.9111 - val_accuracy: 0.8582 - val_loss: 0.4046 - val_precision: 0.8767 - val_recall: 0.8497
Epoch 4/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 107s 10ms/step - accuracy: 0.9243 - loss: 0.1409 - precision: 0.9338 - recall: 0.9171 - val_accuracy: 0.8611 - val_loss: 0.4277 - val_precision: 0.8780 - val_recall: 0.8540
Epoch 5/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 95s 9ms/step - accuracy: 0.9281 - loss: 0.1311 - precision: 0.9355 - recall: 0.9222 - val_accuracy

In [36]:
model_1.save('Models/gru.keras')

In [38]:
model_2.save('Models/lstm.keras')

In [39]:
model_3.save('Models/bigru.keras')

In [40]:
model_4.save('Models/bilstm.keras')